In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.express as px
import matplotlib.pylab as plt
from pandas.api.types import CategoricalDtype
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()
sns.set_style("whitegrid")
# from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold, cross_val_score,train_test_split
from xgboost import XGBClassifier

In [ ]:
df=pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv").drop("id",axis=1)

In [ ]:
df.head(5)

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
df.describe() 

## analysis the effect of categorical data of the target variable

In [ ]:
for i in df.select_dtypes("object"):
    for j in df[i].unique():
        p=(df[df[i]==j].stroke.mean())*100
        print(f"if [{i}]== [{j}] , the probability of have stroke is [{round(p,2)}] %")
    print("-"*60)

## show the variance in stroke by the variables

In [ ]:
f,axes=plt.subplots(nrows=5,ncols=1,figsize=(10,10))
f.suptitle("show the difference")
for i,value in enumerate(df.select_dtypes("object")):
    sns.barplot(x=value,y="stroke",data=df,ax=axes[i])
plt.tight_layout()

In [ ]:
# show the target varialbe
px.pie(data_frame=df["stroke"].value_counts(dropna=False),
       names=df["stroke"].value_counts(dropna=False).index,
       values=df["stroke"].value_counts(dropna=False).values,
       title="Stroke values",
      )


- there is large impalance here

In [ ]:
f,axes=plt.subplots(nrows=1,ncols=2,figsize=(10,10))
f.suptitle("the distribution of float data ")
sns.distplot(x=df["bmi"],color="red",ax=axes[0])
sns.distplot(x=df["avg_glucose_level"],color="green",ax=axes[1])

### start working with data_ for machine learning model

In [ ]:
def load_data(df_train,df_test):
    x=pd.concat([df_train,df_test],axis=0)
    x=clean(x)
    x=encode(x)
    train=x.loc[df_train.index,:] 
    test=x.loc[df_test.index,:]
    return train,test

In [ ]:
# searching for unusul value or replace them to be simple
def clean(df):
    df["smoking_status"]=df["smoking_status"].replace({
         "formerly smoked":"before",
        "never smoked":"never"
    })
    df["work_type"]=df["work_type"].replace({
        "children":"child",
        "Govt_jov":"gov",
        "Never_worked":"never",
        "Self-employed":"self",
        "Private":"private"
    })
    return df 

In [ ]:
# correctly encode the object variables to categories
def encode(df):
    df["smoking_status"]=df["smoking_status"].astype(CategoricalDtype(["Unknown","never","before","smokes"],ordered=True))
    for col in df.select_dtypes(["object"]):
        df[col]=df[col].astype("category")
        if "other" not in df[col].cat.categories:
            df[col].cat.add_categories("other")
    return df 

In [ ]:
def normalize(df):
    df["bmi"]=np.log1p(df["bmi"])
    df["avg_glucose_level"]=np.log1p(df["avg_glucose_level"])
    return df 

In [ ]:
x=df.drop("stroke",axis=1)
y=df["stroke"]


In [ ]:
df_train,df_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=0,stratify=y) 
train,test=load_data(df_train,df_test) 

In [ ]:
def mi(X,y):
    for i in X.select_dtypes(["category"]):
        X[i],_=X[i].factorize()
    discreate_features=[pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    if "bmi" in X.columns:
        X["bmi"].fillna(0,inplace=True)
    scores=mutual_info_classif(X,y,discrete_features=discreate_features,random_state=0)
    series=pd.Series(scores,index=X.columns).sort_values(ascending=False)
    return series

In [ ]:
mi(train,y_train) 

In [ ]:
def score_data(X,y):
    for i in X.select_dtypes(["category"]):
        X[i]=X[i].cat.codes
    scores=cross_val_score(XGBClassifier(use_label_encoder=False,eval_metric="error"),X,y,cv=StratifiedKFold(n_splits=3),scoring="roc_auc")
    print( scores.mean() )

In [ ]:
score_data(train,y_train)

- the intial score is .8

- it seams that [gender ,Residence_type ] hav very low affect on the the target 
- let's try to remove them

In [ ]:
def drop_low_mi(df):
    df=df.drop(["gender","Residence_type"],axis=1)
    return df

In [ ]:
# v1 -> first version 
v1=train.copy()
v1=drop_low_mi(v1)
score_data(v1,y_train)

- it inceares the score, but very little.

### create features

In [ ]:
px.scatter(train ,x="bmi",y="avg_glucose_level")

In [ ]:
px.scatter(train ,x="age",y="avg_glucose_level")

In [ ]:
px.scatter(train,x="age",y="bmi")

- let's try making new feature that cluster the age and the bmi

In [ ]:
# know the best cluster 
ks=[1,2,3,4,5,6,7,8]
inertia=[]
for k in ks:
    kmeans=KMeans(n_clusters=k)
    x=train[["age","bmi","avg_glucose_level"]] 
    kmeans.fit(x)
    inertia.append(kmeans.inertia_)
fig=px.line(x=ks,y=inertia,labels={"x":"number of clusters","y":"the interia"},title="numbers of clusters VS interia")
fig.add_vline(x=3)

- so , i will stick with the 3 clusters as the [ elbow point ]

In [ ]:
def cluster(df):
    kmeans=KMeans(n_clusters=3, n_init=30)
    x=df[["age","bmi","avg_glucose_level"]]
    kmeans.fit(x)
    labels=kmeans.predict(x)
    X=pd.DataFrame()
    X["cluster"]=labels
    return X

In [ ]:
# version 2
v2=train.copy()
v2=drop_low_mi(v2)
v2=v2.join(cluster(v2))
score_data(v2,y_train)

- the cluster was successful

### try to get the varience between featrues with PCA.

In [ ]:
def pca(df):
    data=df[["age","bmi","avg_glucose_level"]]
    pca=PCA()
    x=pca.fit_transform(data)
    
    cols=[f"pca_{i}" for i in range(len(data.columns))]
    X=pd.DataFrame(x,columns=cols)
    coefs=pd.DataFrame(pca.components_.T,index=["age","bmi","avg_glucose_level"],columns=X.columns)
    return X, coefs
    

In [ ]:
X_pca,com=pca(train)
com 

In [ ]:
pca_information=mi(X_pca,y_train)
pca_information

In [ ]:
def new_feature(df):
    X=pd.DataFrame()
    X["age/bmi"]=df["age"]/df["bmi"]
    return X 

In [ ]:
v3=train.copy()
v3=drop_low_mi(v3)
v3=v3.join(cluster(v3))
v3=v3.join(new_feature(v3))
score_data(v2,y_train)

## it doesn't help at all.

In [ ]:
clustering=test[["age","bmi","avg_glucose_level"]]
clustering

In [ ]:
# make a function to get the final data
def get_data(train,test):
    X=train.copy()
    X=drop_low_mi(X)
    X=X.join(cluster(X))
    X["bmi"]=X["bmi"].fillna(0)
    df_test=test.copy()
    df_test["bmi"]=df_test["bmi"].fillna(0)
    df_test=drop_low_mi(df_test)
    clustering=df_test[["age","bmi","avg_glucose_level"]]
    label=kmeans.predict(clustering)
    v=pd.DataFrame()
    v["cluster"]=label
    df_test=df_test.join(v)
    for i in X.select_dtypes(["category"]):
        X[i]=X[i].cat.codes
    for i in df_test.select_dtypes(["category"]):
        df_test[i]=df_test[i].cat.codes
    return X,df_test

In [ ]:
X,X_test=get_data(train,test )

In [ ]:
X.shape,X_test.shape

- X : represents the valid date
- X_test : represents the unseen data [ used to make prediction after fiting the model ]

#### due to the imbalance , the model performance will be so bad in predicting [ stroke=1 ].so , let's try 
- oversampling the minority class.
- under sampling the majority class.
- try to combine both [ under_dampling , over_sampling ]

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report

### what i will do :-
- apply [ over sampling and under sampling ] to the data.
- fit the data to the model and predict the x_test
- make the [ classification_report ] and see the effect in predicting the [ stroke=1]
     - i guess the important calss here is [ stroke=1 ] 
           - predicting the person to have stroke,then go to the hospital to see if it is right. is     better than predicting that he/she has no stroke to try to trea him/here in wrong ways.

In [ ]:
def original_data(train,y_train,test,y_test):
    model= XGBClassifier()
    model.fit(train,y_train)
    y_pred=model.predict(test)
    print(classification_report(y_pred,y_test))

In [ ]:
original_data(X,y_train,X_test,y_test)

In [ ]:
def over_sampling(train,y_train,test,y_test):
    d_train,target=train.copy(),y_train.copy()
    sampling= RandomOverSampler(sampling_strategy=.4,random_state=0)
    over_x,over_y=sampling.fit_resample(d_train,target)
    model= XGBClassifier()
    model.fit(over_x,over_y)
    y_pred=model.predict(test)
    print(classification_report(y_pred,y_test))
    

In [ ]:
over_sampling(X,y_train,X_test,y_test)

In [ ]:
def under_sampling(train,y_train,test,y_test):
    d_train,target=train.copy(),y_train.copy()
    sampling= RandomUnderSampler(sampling_strategy=.4,random_state=0)
    over_x,over_y=sampling.fit_resample(d_train,target)
    model= XGBClassifier()
    model.fit(over_x,over_y)
    y_pred=model.predict(test)
    print(classification_report(y_pred,y_test))

In [ ]:
under_sampling(X,y_train,X_test,y_test)

- it seems that under_sampling did a good work rather than over_sampling
- let's combine them

In [ ]:
def over_under_sampling(train,y_train,test,y_test):
    d_train,target=train.copy(),y_train.copy()
    sampling= RandomOverSampler(sampling_strategy=.3,random_state=0)
    over_x,over_y=sampling.fit_resample(d_train,target)
    sampling= RandomUnderSampler(sampling_strategy=.4,random_state=0)
    over_under_x,over_under_y=sampling.fit_resample(over_x,over_y)
    model= XGBClassifier()
    model.fit(over_under_x,over_under_y)
    y_pred=model.predict(test)
    print(classification_report(y_pred,y_test))

In [ ]:
over_under_sampling(X,y_train,X_test,y_test)

#### as it looks like , under_sampling is the best here   ......... [ for me ]

## if i did somthing wrong or you have any other ideas. please, tel me.
#### thanks